### Introduction

In the following notebook, I will be cleaning a raw data file of review data from Inside Airbnb and concatenating review scores from a listings file.

**Read in libraries**

In [1]:
import numpy as np
import pandas as pd
import swifter

**Read in data**

In [2]:
#Set path to data on local machine
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\01_Raw\SF Airbnb'

#Read in data
reviews = pd.read_csv(path + '/2020_0526_Aggregated_Reviews.csv',
                 parse_dates= ['date'],index_col=0)

#Set path to local machine
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\02_Cleaned'

#Read in data
listings = pd.read_csv(path + '/2020_0520_Listings_Cleaned.csv', index_col=0,
                parse_dates=['host_since','last_review'])

### Data Overview

**Preview Data**

In [3]:
#Display data, print shape
print('Review data shape:', reviews.shape)
display(reviews.head(3))

Review data shape: (466004, 6)


,comments,date,id,listing_id,reviewer_id,reviewer_name
0,"Our experience was, without a doubt, a five st...",2009-07-23,5977,958,15695,Edmund C
1,Returning to San Francisco is a rejuvenating t...,2009-08-03,6660,958,26145,Simon
2,We were very pleased with the accommodations a...,2009-09-27,11519,958,25839,Denis


**View data description**

In [4]:
#View data description
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466004 entries, 0 to 359216
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   comments       465824 non-null  object        
 1   date           466004 non-null  datetime64[ns]
 2   id             466004 non-null  int64         
 3   listing_id     466004 non-null  int64         
 4   reviewer_id    466004 non-null  int64         
 5   reviewer_name  466003 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 24.9+ MB


## Data Preparation

**Subset cols from Listings for merge**

In [5]:
#Subset
listings = listings[['id','review_scores_rating','last_review','host_id']]

#Check
display(listings.head())

,id,review_scores_rating,last_review,host_id
0,958,97.0,2020-03-13,1169
1,5858,98.0,2017-08-06,8904
2,7918,84.0,2020-03-06,21994
3,8142,93.0,2018-09-12,21994
4,8339,97.0,2019-06-28,24215


**Merge Data**

In [6]:
#Merge review and listings data
merged_df = pd.merge(left = reviews, right=listings, how = 'left', left_on=['listing_id','date'], right_on=['id', 'last_review'])

#Remove rows where review_scores_rating is na. Drop unnecessary cols and drop dupes
merged_df = merged_df[~merged_df.review_scores_rating.isna()]
merged_df.drop(['last_review','reviewer_name', 'id_x','id_y'], inplace =True, axis=1)
merged_df.drop_duplicates(inplace=True)

#Strip white space
merged_df['comments'] = merged_df['comments'].str.strip()

#Sort data by date
merged_df.sort_values(by='date', inplace = True)

#Check
display(merged_df.head())

,comments,date,listing_id,reviewer_id,review_scores_rating,host_id
7790,Paul has a super nice place and is a super nic...,2010-10-04,44680,140276,100.0,196626.0
10317,Did not stay here. There was a challenge that ...,2011-05-23,59831,501557,20.0,287859.0
12146,"He's great. Location is perfect, especially if...",2011-09-30,71779,654056,60.0,368770.0
27172,Rebecca's studio is great. I felt completely a...,2011-11-23,261358,1395774,80.0,1257432.0
507880,"Susie is a great hostess, very attentive and a...",2012-02-04,284811,1434864,80.0,1427641.0


## Data Cleaning

**Drop columns not pertaining to NLP analysis**

In [7]:
#Leave comments and review_scores_rating
merged_df.drop(['date','listing_id','reviewer_id','host_id'], axis = 1, inplace = True)

**Resolve missing values**

In [8]:
#View missing values
print('Missing values in merged_df:\n', merged_df.isna().sum())

#Drop NA
merged_df.dropna(inplace=True)

Missing values in merged_df:
 comments                30
review_scores_rating     0
dtype: int64


**Comments mentioning automated review**

In [9]:
#Drop rows mentioning 'This is an automated posting.'
merged_df = merged_df.loc[~(merged_df['comments'].str.contains('This is an automated posting.'))]

#Check shape
merged_df .shape

(39264, 2)

**Language Detector**

Not all reviews are in English and they will require translation. Assigning language of reviews as a column

In [10]:
#Read in library for language detection
from langdetect import detect

#Write function to determine language based on text data
def language_detection(text):
    try:
        return detect(text)
    except:
        return None
    
#Apply language_detection to comments
merged_df['language']=merged_df.comments.swifter.apply(language_detection)

#Drop rows w/o language or are not english
merged_df.dropna(axis = 0, inplace=True)
merged_df = merged_df.loc[merged_df.language == 'en']

#Drop language column
merged_df.drop('language', inplace = True, axis = 1)

#Check
display(merged_df.head())

,comments,review_scores_rating
7790,Paul has a super nice place and is a super nic...,100.0
10317,Did not stay here. There was a challenge that ...,20.0
12146,"He's great. Location is perfect, especially if...",60.0
27172,Rebecca's studio is great. I felt completely a...,80.0
507880,"Susie is a great hostess, very attentive and a...",80.0


### Write CSV

In [11]:
#Print merged_df shape
print("Data shape:", merged_df.shape)

#Set path and write file
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\02_Cleaned'
merged_df.to_csv(path + '/2020_0526_Reviews_Cleaned.csv')

Data shape: (36750, 2)
